In [8]:
import numpy as np
import seaborn as sns

from matplotlib import pyplot as plt
from skimage.restoration import estimate_sigma
from src.utilities import window_pow_db, get_bayes_denoised, get_visu_denoised
from src.utilities import pow_c_a_dwt, pow_c_d_dwt, logistic_map
from src.communications_module import OFDMModulator
from src.communications_module.channels import AWGNChannel, FadingChannel, TDLChannel

In [9]:
rx_snrs_list = [-15, -5, 0, 5, 15]
fft_size = 512
subcarriers = 128
num_pilots = 8
bits_per_sym = 4
cp_ratio = 4
num_symbols = 50

ch_type = 'rayleigh'
discrete_del = [5.2e-6, 6.8e-6, 7.9e-6, 8.8e-6, 9.9e-6, 13e-6]
avg_path_g=[-11, -12, -7, -8, -9, -10]
max_doppler=0.01
k_factors=[1.5]

In [10]:
import pandas as pd

def averaged_sigma(x):
    return estimate_sigma(x, average_sigmas=True)

In [11]:

for rx_snr in rx_snrs_list:
    df = pd.DataFrame()
#     plt.figure(figsize=(15, 10))
    ofdm_gen = OFDMModulator(bits_per_sym=bits_per_sym, 
                             fft_size=fft_size,
                             subcarriers=subcarriers, 
                             cp_ratio_numitor=cp_ratio, 
                             num_pilots=num_pilots)
    ofdm_sign = ofdm_gen.generate_ofdm_tx_signal(num_symbols, continuous_transmission=False)

    fading_channel = FadingChannel(x_in=ofdm_sign,
                                   channel_type=ch_type,
                                   discrete_path_delays=discrete_del,
                                   avg_path_gains=avg_path_g,
                                   max_doppler_shift=max_doppler,
                                   k_factors=k_factors)
    
#     fading_channel = TDLChannel(ofdm_sign, num_taps=1)
    y_out_faded = fading_channel.filter_x_in()
#     plt.plot(abs(ofdm_sign), label='Original')
#     plt.plot(abs(y_out_faded), label='RX')
#     plt.title(f'SNR: {rx_snr}')
#     plt.ylim((0,2))
#     plt.legend()
#     fading_channel.plot_power_delay_profile()
    
    noise_model = AWGNChannel(y_out_faded, rx_snr)
    y_out_noisy, n = noise_model.filter_x_in()
#     plt.plot(abs(y_out_noisy), label='RX_faded&noisy')
#     plt.title(f'SNR: {rx_snr}')
#     plt.ylim((0,25))
#     plt.plot(abs(n))
#     plt.plot(abs(y_out_noisy))
#     plt.plot(abs(ofdm_sign), label='T')
#     plt.title(f'SNR: {rx_snr}')
#     plt.legend()
    
    OFDM_RX, noise = y_out_noisy, n
    df['TX_OFDM'] = abs(ofdm_sign)
    df['noise'] = abs(noise)
    df['RX_OFDM'] = abs(OFDM_RX)
    df['RE_RX_OFDM'] = OFDM_RX.real
    df['IM_RX_OFDM'] = OFDM_RX.imag
    df['RX_BAYES'] = get_bayes_denoised(df['RX_OFDM'], df['noise'])
    df['RX_VISU'] = get_visu_denoised(df['RX_OFDM'], df['noise'])
#     df['AVG_CA_RXDWT'] = df['RX_VISU'].rolling(100).apply(pow_c_a_dwt)
#     df['AVG_CD_RXDWT'] = df['RX_VISU'].rolling(100).apply(pow_c_d_dwt)
#     df['LOGISTICM_POW'] = df['RX_BAYES'].rolling(50).apply(logistic_map)
    df['RX_POW_DB'] = df['RX_BAYES'].rolling(50).apply(window_pow_db)
    
#     df['RX_ENTROPY'] = df['RX_OFDM'].rolling(100).apply(shannon_entropy)
    df['sigma'] = df['noise'].rolling(50).apply(np.var)
    
    last_nan = np.where(np.asanyarray(np.isnan(df)))[0][-1]
    
#     df = df.fillna(0)
    df = df.loc[(last_nan+1):, :]
    
    
    ones = df['TX_OFDM'] > 0
    df.loc[ones, 'USER'] = 1
    df.loc[~ones, 'USER'] = 0
    if len(discrete_del) == 1:
        df.to_csv(f'E:/Faculta/Anul2/Disertatie/cognitive-aether/datasets/final/{rx_snr}SNR_{ch_type}flat.csv', 
                  index=False)
    else:
        df.to_csv(f'E:/Faculta/Anul2/Disertatie/cognitive-aether/datasets/final/{rx_snr}SNR_{ch_type}_fselective_ofdm.csv', 
                  index=False)

Required signal power: 0.06329957210796941 [W]=[V^2]
Initial signal power: 0.003031560747586292 [W]=[V^2]
Signal amplitude rescale factor: 4.569484931646892 [Volts]
RX Signal power: 0.0633. Noise power: 2.0017, SNR [dB]: -15.0000
Required signal power: 0.6324469221416247 [W]=[V^2]
Initial signal power: 0.0008840063640605926 [W]=[V^2]
Signal amplitude rescale factor: 26.74757080959761 [Volts]
RX Signal power: 0.6324. Noise power: 2.0000, SNR [dB]: -5.0000
Required signal power: 1.9901708409014505 [W]=[V^2]
Initial signal power: 0.004419424124327904 [W]=[V^2]
Signal amplitude rescale factor: 21.220828639921578 [Volts]
RX Signal power: 1.9902. Noise power: 1.9902, SNR [dB]: 0.0000
Required signal power: 6.2834876625114235 [W]=[V^2]
Initial signal power: 0.007362820471538998 [W]=[V^2]
Signal amplitude rescale factor: 29.213142540027306 [Volts]
RX Signal power: 6.2835. Noise power: 1.9870, SNR [dB]: 5.0000
Required signal power: 63.621697280423234 [W]=[V^2]
Initial signal power: 0.003107911